<a href="https://colab.research.google.com/github/BenUCL/Reef-acoustics-and-AI/blob/main/Code/Feature_extraction_with_compound_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Compound index calculation**



Connect your Google drive so you can access uploaded data:

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Install scikit-maad package  (https://scikit-maad.github.io/)

In [1]:
!pip3 install scikit-maad

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 134 kB 8.6 MB/s 


Import packages

In [3]:
import pandas as pd
import os
from maad import sound, features, util
from maad.util import (date_parser, plot_correlation_map,
                       plot_features_map, plot_features, false_Color_Spectro)

from statistics import mean
from statistics import stdev
import numpy as np 

# may be some redundant imports
from datetime import datetime
from datetime import timedelta
from datetime import time
from time import strftime
import time

Set locations of audio and where to save output

In [24]:
# Path to the location where your audio file are stored:
audio_dir = date_parser(r"/content/drive/MyDrive/Reef soundscapes with AI/audio_dir/")

results_directory = r'/content/drive/MyDrive/Reef soundscapes with AI/Results'

Run the feature extraction

In [25]:
# define the function to extract features

def index_loops(audio_files, filenames, slice_names, NDSI_fullband,
                #0.96s values
                fish_ACI, fish_ADI, fish_BI, fish_H, fish_Ht, fish_Hf, fish_M,
                shrimp_ACI, shrimp_ADI, shrimp_BI, shrimp_H, shrimp_Ht, shrimp_Hf, shrimp_M,
                full_ACI, full_ADI, full_BI, full_H, full_Ht, full_Hf, full_M,
                #1min averaged values
                fish_ACI_1min_mean, fish_ADI_1min_mean, fish_BI_1min_mean, fish_H_1min_mean,
                fish_Ht_1min_mean, fish_Hf_1min_mean, fish_M_1min_mean,
                shrimp_ACI_1min_mean, shrimp_ADI_1min_mean, shrimp_BI_1min_mean, shrimp_H_1min_mean,
                shrimp_Ht_1min_mean, shrimp_Hf_1min_mean, shrimp_M_1min_mean,
                full_ACI_1min_mean, full_ADI_1min_mean, full_BI_1min_mean, full_H_1min_mean,
                full_Ht_1min_mean, full_Hf_1min_mean, full_M_1min_mean,
                NDSI_1min_mean,
                fish_ACI_1min_std, fish_ADI_1min_std, fish_BI_1min_std, fish_H_1min_std,
                fish_Ht_1min_std, fish_Hf_1min_std, fish_M_1min_std,
                shrimp_ACI_1min_std, shrimp_ADI_1min_std, shrimp_BI_1min_std, shrimp_H_1min_std,
                shrimp_Ht_1min_std, shrimp_Hf_1min_std, shrimp_M_1min_std,
                full_ACI_1min_std, full_ADI_1min_std, full_BI_1min_std, full_H_1min_std,
                full_Ht_1min_std, full_Hf_1min_std, full_M_1min_std,
                NDSI_1min_std,
                mean_slice_names):
    
    df = audio_files

    #create lists that will temporarily store index results from which the mean can be calculated, then reset
    fish_ACI_temp = []
    fish_ADI_temp = []
    fish_BI_temp = []
    fish_H_temp = []
    fish_Ht_temp = []
    fish_Hf_temp = []
    fish_M_temp = []

    shrimp_ACI_temp = []
    shrimp_ADI_temp = []
    shrimp_BI_temp = []
    shrimp_H_temp = []
    shrimp_Ht_temp = [] 
    shrimp_Hf_temp = []
    shrimp_M_temp = []
    
    full_ACI_temp = []
    full_ADI_temp = []
    full_BI_temp = []
    full_H_temp = []
    full_Ht_temp = []
    full_Hf_temp = []
    full_M_temp = []

    NDSI_temp = []
    
    for index, row in df.iterrows() :
        """ The following loop completes these tasks: load audio file, slice into 0.96s segments, append the time to each 
        slice with 0.96s added to end of the file name each slice, filter each segment into fish & shrimp band, 
        compute the spectrogram for indices that require this, compute all indices for each segment in both bands, 
        append the results to lists which are combined into a dataframe after the loop"""

        # get the full filename of the corresponding row
        fullfilename = row['file']
        # Save file basename
        path, filename = os.path.split(fullfilename)

        #add file name to list for results dataframe
        filenames.append(filename)

        #rip timestamp
        #t = filename[-16:-4:]#####################################################################################
        t2 = filename.split(".")[1][0:4]
        t1 = filename.split(".")[3]
        t = t1+t2+'00'
        recording_start_time = pd.to_datetime(t, format='%y%m%d%H%M%S')
        mean_slices_time = recording_start_time - timedelta(minutes=1) #for 1min average    

        #print file name being iterated upon
        print ('\n**************************************************************')
        print (filename)

        #### Load the original sound (16bits) and get the sampling frequency fs
        try :
            wave, fs = sound.load(filename=fullfilename, channel='left', detrend=True, verbose=False)

        except:
            # Delete the row if the file does not exist or raise a value error (i.e. no EOF)
            df.drop(index, inplace=True)
            continue

        #divide length of audio file by 0.96s = total_segs, round down to nearest integer, then do a for loop that goes from
         #0:total_segs. This loop should do the index to each 

        #get the length of audio file in seconds using number of samples/sample rate
        total_length = len(wave)/fs

        #get number of 0.96s segs in file, rounding down to nearest integer so there is no partial seg at the end
        number_segs = int(total_length/0.96) #round

        #0.96s time keeping
        start = 0
        end = 0.96
        zero_time = recording_start_time - timedelta(milliseconds=960) #zero_time - timedelta(milliseconds=960) #so that the first iteration resets to 0

        #counter for averaging
        count = 0 

        for k in range (number_segs): #should this be number_segs-1? #don't think so
            sliced_file = sound.trim(wave, fs, min_t = start, max_t = end)
            start = round(start + 0.96, 2)
            end = round(end + 0.96, 2)

            #naming of slices using time   ##### Make some changes here to make te 0.96s segments names neater
            zero_time = zero_time + timedelta(milliseconds=960)
            string_time = zero_time.strftime('%H.%M.%S.%f')[:-4]
            track_name = filename[:-4] + 'T' + string_time + '.wav'
            slice_names.append(track_name) 


            ####### fish band
            #filter track
            filtered = sound.select_bandwidth(x = sliced_file, fs = fs, fcut = [50, 2000], forder = 5,
                                fname = 'butter', ftype = 'bandpass')

            #compute spectrogram for: ACI, ADI, Hf
            Sxx, tn, fn, ext = sound.spectrogram (filtered, fs, nperseg = 256)  

            #ACI
            _, _ , ACI_fish = features.acoustic_complexity_index(Sxx)

            #ADI, uses bins 10 bins, each 1/10th of the full freq range
            ADI_fish = features.acoustic_diversity_index(Sxx, fn, fmin = 50, fmax = 2000, bin_step = 190, dB_threshold = -50)

            #BI
            BI_fish = features.bioacoustics_index(Sxx, fn, flim = (50, 2000))

            #H
            H_fish = util.entropy(filtered)

            #Ht
            #Ht_fish = features.temporal_entropy(filtered, Nt = 256)

            #Hf
            Hf_fish, Ht_per_bin = features.frequency_entropy(Sxx)

            #M, amplitude index
            M_fish = features.temporal_median(filtered, Nt = 256)

            #append results to lists created earlier
            fish_ACI.append(ACI_fish) 
            fish_ADI.append(ADI_fish) 
            fish_BI.append(BI_fish)
            fish_H.append(H_fish) 
            #fish_Ht.append(Ht) 
            fish_Hf.append(Hf_fish)
            fish_M.append(M_fish) 




            ####### shrimp band
            #filter track
            filtered = sound.select_bandwidth(x = sliced_file, fs = fs, fcut = [2000, 8000], forder = 5,
                                    fname = 'butter', ftype = 'bandpass')

            #compute spectrogram for: ACI, ADI, Hf
            Sxx, tn, fn, ext = sound.spectrogram (filtered, fs, nperseg = 256)  

            #ACI
            _, _ , ACI_shrimp = features.acoustic_complexity_index(Sxx)

            #ADI, uses bins 10 bins, each 1/10th of the full freq range
            ADI_shrimp = features.acoustic_diversity_index(Sxx,fn, fmin = 2000, fmax = 8000, bin_step = 600, dB_threshold = -50)

            #BI
            BI_shrimp = features.bioacoustics_index(Sxx,fn,flim = (2000, 8000))

            #H
            H_shrimp = util.entropy(filtered)

            #Ht
            #Ht_shrimp = features.temporal_entropy(filtered, Nt = 256)

            #Hf
            Hf_shrimp, Ht_per_bin = features.frequency_entropy(Sxx)

            #M, amplitude index
            M_shrimp = features.temporal_median(filtered, Nt = 256)

            #append results to lists created earlier
            shrimp_ACI.append(ACI_shrimp) 
            shrimp_ADI.append(ADI_shrimp) 
            shrimp_BI.append(BI_shrimp)
            shrimp_H.append(H_shrimp) 
            #shrimp_Ht.append(Ht_shrimp) 
            shrimp_Hf.append(Hf_shrimp) 
            shrimp_M.append(M_shrimp) 
            
            
            ####### full band
            #filter track
            filtered = sound.select_bandwidth(x = sliced_file, fs = fs, fcut = [50, 8000], forder = 5,
                                fname = 'butter', ftype = 'bandpass')

            #compute spectrogram for: ACI, ADI, Hf
            Sxx, tn, fn, ext = sound.spectrogram (filtered, fs, nperseg = 256)  

            #ACI
            _, _ , ACI_full = features.acoustic_complexity_index(Sxx)

            #ADI, uses bins 10 bins, each 1/10th of the full freq range
            ADI_full = features.acoustic_diversity_index(Sxx, fn, fmin = 50, fmax = 8000, bin_step = 190, dB_threshold = -50)

            #BI
            BI_full = features.bioacoustics_index(Sxx, fn, flim = (50, 8000))

            #H
            H_full = util.entropy(filtered)

            #Ht
            #Ht_full = features.temporal_entropy(filtered, Nt = 256)

            #Hf
            Hf_full, Ht_per_bin = features.frequency_entropy(Sxx)

            #M, amplitude index
            M_full = features.temporal_median(filtered, Nt = 256)

            #append results to lists created earlier
            full_ACI.append(ACI_full) 
            full_ADI.append(ADI_full) 
            full_BI.append(BI_full)
            full_H.append(H_full) 
            #full_Ht.append(Ht_full) 
            full_Hf.append(Hf_full)
            full_M.append(M_full) 

            

            #compute NDSI 
            #running without reapplying a filter massively slows down the script, unsure why? 
            filtered = sound.select_bandwidth(x = sliced_file, fs = fs, fcut = [50, 20000], forder = 5,
                                    fname = 'butter', ftype = 'bandpass')
            Sxx, tn, fn, ext = sound.spectrogram (sliced_file, fs, nperseg = 256)
            NDSI, ratioBA, antroPh, bioPh  = features.soundscape_index(Sxx, fn, flim_bioPh = (2000, 8000), flim_antroPh = (50, 2000))
            NDSI_fullband.append(NDSI) 

            #store averages for 1 min
              #could maybe put these lists into a master list and execute a commond over each value of this to make code neater
            #mean_slices_time = zero_time1 #+ timedelta(milliseconds=960) #0.96s was subtracted from zero_time ################################################### do i need this line?

            if count == 61: #0.96s goes into 1min 62 times, adjust depending on length of time wanted
                #take the average of the slices for the length of time period chosen
                fish_ACI_1min_mean.append(mean(fish_ACI_temp)) 
                fish_ADI_1min_mean.append(mean(fish_ADI_temp)) 
                fish_BI_1min_mean.append(mean(fish_BI_temp)) 
                fish_H_1min_mean.append(mean(fish_H_temp)) 
                #fish_Ht_1min_mean.append(mean(fish_Ht_temp)) 
                fish_Hf_1min_mean.append(mean(fish_Hf_temp)) 
                fish_M_1min_mean.append(mean(fish_M_temp)) 

                shrimp_ACI_1min_mean.append(mean(shrimp_ACI_temp)) 
                shrimp_ADI_1min_mean.append(mean(shrimp_ADI_temp)) 
                shrimp_BI_1min_mean.append(mean(shrimp_BI_temp)) 
                shrimp_H_1min_mean.append(mean(shrimp_H_temp)) 
                #shrimp_Ht_1min_mean.append(mean(shrimp_Ht_temp)) 
                shrimp_Hf_1min_mean.append(mean(shrimp_Hf_temp)) 
                shrimp_M_1min_mean.append(mean(shrimp_M_temp)) 
                
                full_ACI_1min_mean.append(mean(full_ACI_temp)) 
                full_ADI_1min_mean.append(mean(full_ADI_temp)) 
                full_BI_1min_mean.append(mean(full_BI_temp)) 
                full_H_1min_mean.append(mean(full_H_temp)) 
                #full_Ht_1min_mean.append(mean(full_Ht_temp)) 
                full_Hf_1min_mean.append(mean(full_Hf_temp)) 
                full_M_1min_mean.append(mean(full_M_temp)) 

                NDSI_1min_mean.append(mean(NDSI_temp)) 
                
                #standard dev
                fish_ACI_1min_std.append(stdev(fish_ACI_temp)) 
                fish_ADI_1min_std.append(stdev(fish_ADI_temp)) 
                fish_BI_1min_std.append(stdev(fish_BI_temp)) 
                fish_H_1min_std.append(stdev(fish_H_temp)) 
                #fish_Ht_1min_std.append(stdev(fish_Ht_temp)) 
                fish_Hf_1min_std.append(stdev(fish_Hf_temp)) 
                fish_M_1min_std.append(stdev(fish_M_temp)) 

                shrimp_ACI_1min_std.append(stdev(shrimp_ACI_temp)) 
                shrimp_ADI_1min_std.append(stdev(shrimp_ADI_temp)) 
                shrimp_BI_1min_std.append(stdev(shrimp_BI_temp)) 
                shrimp_H_1min_std.append(stdev(shrimp_H_temp)) 
                #shrimp_Ht_1min_std.append(stdev(shrimp_Ht_temp)) 
                shrimp_Hf_1min_std.append(stdev(shrimp_Hf_temp)) 
                shrimp_M_1min_std.append(stdev(shrimp_M_temp)) 
                
                full_ACI_1min_std.append(stdev(full_ACI_temp)) 
                full_ADI_1min_std.append(stdev(full_ADI_temp)) 
                full_BI_1min_std.append(stdev(full_BI_temp)) 
                full_H_1min_std.append(stdev(full_H_temp)) 
                #full_Ht_1min_std.append(stdev(full_Ht_temp)) 
                full_Hf_1min_std.append(stdev(full_Hf_temp)) 
                full_M_1min_std.append(stdev(full_M_temp)) 

                #NDSI_1min_std.append(stdev(NDSI_temp)) #this is causing an error for no reason, S/O says its a source code bug
                NDSI_1min_std.append(np.std(NDSI_temp))

                #reset lists to blank and counter to zero
                fish_ACI_temp = []
                fish_ADI_temp = []
                fish_BI_temp = []
                fish_H_temp = []
                #fish_Ht_temp = []
                fish_Hf_temp = []
                fish_M_temp = []

                shrimp_ACI_temp = []
                shrimp_ADI_temp = []
                shrimp_BI_temp = []
                shrimp_H_temp = []
                #shrimp_Ht_temp = [] 
                shrimp_Hf_temp = []
                shrimp_M_temp = []
                
                full_ACI_temp = []
                full_ADI_temp = []
                full_BI_temp = []
                full_H_temp = []
                #full_Ht_temp = []
                full_Hf_temp = []
                full_M_temp = []

                NDSI_temp = []

                count = 0
                mean_slices_time = mean_slices_time + timedelta(minutes=1) #for 1min average  
                string_time1 = mean_slices_time.strftime('%H.%M.%S')
                mean_track_name = filename[:-4] + 'T' + string_time1 + '.wav'
                #mean_slice_names.append(mean_track_name)   ######### changed for using presliced files
                mean_slice_names.append(filename) 

            else:
                count = count + 1

                fish_ACI_temp.append(ACI_fish) 
                fish_ADI_temp.append(ADI_fish) 
                fish_BI_temp.append(BI_fish)
                fish_H_temp.append(H_fish) 
                #fish_Ht_temp.append(Ht_fish) 
                fish_Hf_temp.append(Hf_fish)
                fish_M_temp.append(M_fish) 

                shrimp_ACI_temp.append(ACI_shrimp) 
                shrimp_ADI_temp.append(ADI_shrimp) 
                shrimp_BI_temp.append(BI_shrimp)
                shrimp_H_temp.append(H_shrimp) 
                #shrimp_Ht_temp.append(Ht_shrimp) 
                shrimp_Hf_temp.append(Hf_shrimp) 
                shrimp_M_temp.append(M_shrimp)
                
                full_ACI_temp.append(ACI_full) 
                full_ADI_temp.append(ADI_full) 
                full_BI_temp.append(BI_full)
                full_H_temp.append(H_full) 
                #full_Ht_temp.append(Ht_full) 
                full_Hf_temp.append(Hf_full)
                full_M_temp.append(M_full) 

                NDSI_temp.append(NDSI)    

In [26]:
#calculate indices for all files in 'audio_directory', which end in .wav

#initiate empty lists for 0.96s results, these will later be stored in a df
filenames = []
slice_names = []

fish_ACI = []
fish_ADI = []
fish_BI = []
fish_H = []
fish_Ht = []
fish_Hf = []
fish_M = []
fish_H1 = []

shrimp_ACI = []
shrimp_ADI = []
shrimp_BI = []
shrimp_H = []
shrimp_Ht = []
shrimp_Hf = []
shrimp_M = []
shrimp_H1 = []

full_ACI = []
full_ADI = []
full_BI = []
full_H = []
full_Ht = []
full_Hf = []
full_M = []
full_H1 = []

NDSI_fullband = [] #NDSI compares sound between fish and shrimp band, hence stands alone


#initiate empty lists which will store the index results averaged over a longer timeframe (e.g 1min)
fish_ACI_1min_mean = []
fish_ADI_1min_mean = []
fish_BI_1min_mean = []
fish_H_1min_mean = []
fish_Ht_1min_mean = []
fish_Hf_1min_mean = []
fish_M_1min_mean = []

shrimp_ACI_1min_mean = []
shrimp_ADI_1min_mean = []
shrimp_BI_1min_mean = []
shrimp_H_1min_mean = []
shrimp_Ht_1min_mean = [] 
shrimp_Hf_1min_mean = []
shrimp_M_1min_mean = []

full_ACI_1min_mean = []
full_ADI_1min_mean = []
full_BI_1min_mean = []
full_H_1min_mean = []
full_Ht_1min_mean = []
full_Hf_1min_mean = []
full_M_1min_mean = []

NDSI_1min_mean = []

fish_ACI_1min_std = []
fish_ADI_1min_std = []
fish_BI_1min_std = []
fish_H_1min_std = []
fish_Ht_1min_std = []
fish_Hf_1min_std = []
fish_M_1min_std = []

shrimp_ACI_1min_std = []
shrimp_ADI_1min_std = []
shrimp_BI_1min_std = []
shrimp_H_1min_std = []
shrimp_Ht_1min_std = [] 
shrimp_Hf_1min_std = []
shrimp_M_1min_std = []

full_ACI_1min_std = []
full_ADI_1min_std = []
full_BI_1min_std = []
full_H_1min_std = []
full_Ht_1min_std = []
full_Hf_1min_std = []
full_M_1min_std = []

NDSI_1min_std = []

mean_slice_names = []

#execute the index_loops function using the lists from above 
index_loops(audio_dir, filenames, slice_names, NDSI_fullband,
                #0.96s values
                fish_ACI, fish_ADI, fish_BI, fish_H, fish_Ht, fish_Hf, fish_M,
                shrimp_ACI, shrimp_ADI, shrimp_BI, shrimp_H, shrimp_Ht, shrimp_Hf, shrimp_M,
                full_ACI, full_ADI, full_BI, full_H, full_Ht, full_Hf, full_M,
                #1min averaged values
                fish_ACI_1min_mean, fish_ADI_1min_mean, fish_BI_1min_mean, fish_H_1min_mean,
                fish_Ht_1min_mean, fish_Hf_1min_mean, fish_M_1min_mean,
                shrimp_ACI_1min_mean, shrimp_ADI_1min_mean, shrimp_BI_1min_mean, shrimp_H_1min_mean,
                shrimp_Ht_1min_mean, shrimp_Hf_1min_mean, shrimp_M_1min_mean, 
                full_ACI_1min_mean, full_ADI_1min_mean, full_BI_1min_mean, full_H_1min_mean,
                full_Ht_1min_mean, full_Hf_1min_mean, full_M_1min_mean,
                NDSI_1min_mean,
                fish_ACI_1min_std, fish_ADI_1min_std, fish_BI_1min_std, fish_H_1min_std,
                fish_Ht_1min_std, fish_Hf_1min_std, fish_M_1min_std,
                shrimp_ACI_1min_std, shrimp_ADI_1min_std, shrimp_BI_1min_std, shrimp_H_1min_std,
                shrimp_Ht_1min_std, shrimp_Hf_1min_std, shrimp_M_1min_std, 
                full_ACI_1min_std, full_ADI_1min_std, full_BI_1min_std, full_H_1min_std,
                full_Ht_1min_std, full_Hf_1min_std, full_M_1min_std,
                NDSI_1min_std,
                mean_slice_names)


**************************************************************
BaF1.1055H.1678278701.180827.3.35.wav

**************************************************************
BaF3.0915H.1678278701.180829.4.52.wav

**************************************************************
BaF3.0533H.1677983769.180829.3.22.wav

**************************************************************
BaF2.2333H.1677983769.180828.1.15.wav

**************************************************************
BaF3.0533H.1677983769.180829.5.59.wav

**************************************************************
BaF2.1733H.1677983769.180828.3.26.wav

**************************************************************
BaF3.0533H.1677983769.180829.4.53.wav

**************************************************************
BaF1.1055H.1678278701.180827.1.1.wav

**************************************************************
BaF3.0915H.1678278701.180829.1.20.wav

**************************************************************
BaF1.1055H.16782787

In [27]:
#save results from 0.96s slice lists to df
# Ht and AR were removed as they highly correlate with Hf (see: Williams et al (2022). Ecological Indicators)
index_results = pd.DataFrame(
    {'slice': slice_names,
     
     'fish_ACI': fish_ACI,
     'fish_ADI': fish_ADI,
     'fish_H': fish_H,
     #'fish_Ht': fish_Ht,
     'fish_Hf': fish_Hf,
     'fish_M': fish_M,
     'fish_BI': fish_BI,
     #'fish_AR': fish_AR,
     
     'shrimp_ACI': shrimp_ACI,
     'shrimp_ADI': shrimp_ADI,
     'shrimp_BI': shrimp_BI,
     'shrimp_H': shrimp_H,
     #'shrimp_Ht': shrimp_Ht,
     'shrimp_Hf': shrimp_Hf,
     'shrimp_M': shrimp_M,
     #'shrimp_AR': shrimp_AR,
     
     'full_ACI': full_ACI,
     'full_ADI': full_ADI,
     'full_H': full_H,
     #'full_Ht': full_Ht,
     'full_Hf': full_Hf,
     'full_M': full_M,
     'full_BI': full_BI,
     #'full_AR': full_AR,
     
     'NDSI': NDSI_fullband,
    })

#save results from 1min mean lists to df
mean_index_results = pd.DataFrame(
    {'minute': mean_slice_names, #this needs fixing to record time like the 1s stuff does
     
     'fish_ACI': fish_ACI_1min_mean,
     'fish_ACI_std': fish_ACI_1min_std,
     'fish_ADI': fish_ADI_1min_mean,
     'fish_ADI_std': fish_ADI_1min_std,
     'fish_H': fish_H_1min_mean,
     'fish_H_std': fish_H_1min_std,
     #'fish_Ht': fish_Ht_1min_mean,
     #'fish_Ht_std': fish_Ht_1min_std,
     'fish_Hf': fish_Hf_1min_mean,
     'fish_Hf_std': fish_Hf_1min_std,
     'fish_M': fish_M_1min_mean,
     'fish_M_std': fish_M_1min_std,
     'fish_BI': fish_BI_1min_mean,
     'fish_BI_std': fish_BI_1min_std,
     #'fish_AR': fish_AR,
     #'fish_AR_std': fish_AR,
     
     'shrimp_ACI': shrimp_ACI_1min_mean,
     'shrimp_ACI_std': shrimp_ACI_1min_std,
     'shrimp_ADI': shrimp_ADI_1min_mean,
     'shrimp_ADI_std': shrimp_ADI_1min_std,
     'shrimp_BI': shrimp_BI_1min_mean,
     'shrimp_BI_std': shrimp_BI_1min_std,
     'shrimp_H': shrimp_H_1min_mean,
     'shrimp_H_std': shrimp_H_1min_std,
     #'shrimp_Ht': shrimp_Ht_1min_mean,
     #'shrimp_Ht_std': shrimp_Ht_1min_std,
     'shrimp_Hf': shrimp_Hf_1min_mean,
     'shrimp_Hf_std': shrimp_Hf_1min_std,
     'shrimp_M': shrimp_M_1min_mean,
     'shrimp_M_std': shrimp_M_1min_std,
     #'shrimp_AR': shrimp_AR,
     #'shrimp_A_stdR': shrimp_AR,
     
     'full_ACI': full_ACI_1min_mean,
     'full_ACI_std': full_ACI_1min_std,
     'full_ADI': full_ADI_1min_mean,
     'full_ADI_std': full_ADI_1min_std,
     'full_H': full_H_1min_mean,
     'full_H_std': full_H_1min_std,
     #'full_Ht': fish_Ht_1min_mean,
     #'full_Ht_std': fish_Ht_1min_std
     'full_Hf': full_Hf_1min_mean,
     'full_Hf_std': full_Hf_1min_std,
     'full_M': full_M_1min_mean,
     'full_M_std': full_M_1min_std,
     'full_BI': full_BI_1min_mean,
     'full_BI_std': full_BI_1min_std,
     #'fish_AR': fish_AR,
     #'fish_AR_std': fish_AR,
     
     'NDSI': NDSI_1min_mean,
     'NDSI_std': NDSI_1min_std,     
    })



In [28]:
#view results dataframe for 1min time segments
mean_index_results

,minute,fish_ACI,fish_ACI_std,fish_ADI,fish_ADI_std,fish_H,fish_H_std,fish_Hf,fish_Hf_std,fish_M,...,full_H,full_H_std,full_Hf,full_Hf_std,full_M,full_M_std,full_BI,full_BI_std,NDSI,NDSI_std
0,BaF1.1055H.1678278701.180827.3.35.wav,179.151063,18.961872,1.950426,0.105603,0.999767,0.000140,0.426114,0.053091,0.000223,...,0.999955,0.000027,0.734679,0.035224,0.000693,0.000097,4.024393,1.235250,0.477852,0.209203
1,BaF3.0915H.1678278701.180829.4.52.wav,175.021631,18.409587,1.878488,0.102980,0.999725,0.000187,0.434228,0.046118,0.000284,...,0.999956,0.000019,0.744487,0.024361,0.000716,0.000095,3.514929,0.862764,0.446936,0.131809
2,BaF3.0533H.1677983769.180829.3.22.wav,189.423700,20.427394,2.084745,0.398005,0.999795,0.000117,0.464792,0.045987,0.000395,...,0.999938,0.000038,0.736735,0.020284,0.002424,0.000300,5.007314,1.137763,0.563504,0.107107
3,BaF2.2333H.1677983769.180828.1.15.wav,186.161482,22.008563,2.231170,0.045925,0.999873,0.000069,0.490231,0.022307,0.000218,...,0.999943,0.000028,0.732326,0.013501,0.001586,0.000202,6.062306,1.044721,0.613021,0.087742
4,BaF3.0533H.1677983769.180829.5.59.wav,185.030646,17.139834,1.960829,0.377945,0.999742,0.000149,0.435002,0.059959,0.000353,...,0.999947,0.000025,0.743335,0.019465,0.001386,0.000156,3.921846,0.925099,0.494553,0.133031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,BaN11.1731H.805322778.180906.3.24.wav,186.770614,21.097116,2.148827,0.401972,0.999794,0.000160,0.501338,0.030066,0.000274,...,0.999940,0.000039,0.745788,0.014531,0.002069,0.000202,4.991630,0.888435,0.698701,0.124015
148,BaF2.1733H.1677983769.180828.4.28.wav,182.389377,21.470082,2.106955,0.092099,0.999745,0.000204,0.448549,0.047137,0.000407,...,0.999942,0.000032,0.742380,0.022446,0.001658,0.000163,4.129405,0.795918,0.495331,0.157821
149,BaF2.2333H.1677983769.180828.4.48.wav,191.656233,23.325711,2.192312,0.288283,0.999863,0.000081,0.491438,0.023222,0.000247,...,0.999940,0.000029,0.733859,0.019209,0.001714,0.000228,5.757226,1.258642,0.597957,0.083990
150,BaN12.0529H.805322778.180907.4.50.wav,210.007110,15.982194,2.071581,0.556278,0.999897,0.000055,0.472316,0.032458,0.000350,...,0.999938,0.000046,0.728021,0.023292,0.002722,0.000294,6.207309,1.593121,0.443741,0.136824


In [31]:
#save a timestamped csv with 1min results
now = datetime.now()
time_now = now.strftime("%H.%M.%S")
mean_index_results.to_csv(results_directory + '/compound_index_features' + time_now + '.csv')